# FineTune LLM - QLoRA

In [ ]:
"""
Self-contained QLoRA fine-tuning demo on TinyLlama-1.1B-Chat.
"""
import os
import torch
from datasets import Dataset, load_metric
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
)
from peft import LoraConfig, get_peft_model, PeftModel, prepare_model_for_kbit_training
from bitsandbytes import BitsAndBytesConfig

################################################################################
# 0. Environment & constants
################################################################################
MODEL_ID = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
USE_8BIT = True                 # flip to False for full-precision CPU run
HF_PUSH_REPO = "YOUR_USERNAME/tinyllama-lora-demo"   # change to your org/repo
MAX_SEQ_LEN = 256

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Running on {device}. 8-bit: {USE_8BIT}")

In [ ]:
class QLoRAFineTuner:
    """
    Wraps simple QLoRA fine-tuning behind an easy interface so it mirrors
    DistillationFineTuner.
    """
    def __init__(
        self,
        base_model_id: str,
        train_dataset,
        eval_dataset,
        rank: int = 16,
        alpha: int = 32,
        dropout: float = 0.05,
        **training_args_kwargs
    ):
        self.tokenizer = AutoTokenizer.from_pretrained(base_model_id)
        self.tokenizer.pad_token = self.tokenizer.eos_token

        bnb_cfg = BitsAndBytesConfig(   # Different from LoRA
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_type="bf16",
            bnb_4bit_use_double_quant=True,
        )

        model = AutoModelForCausalLM.from_pretrained(   # Different from LoRA
            base_model_id,
            quantization_config=bnb_cfg,
            device_map="auto"
        ).gradient_checkpointing_enable()
        
        model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True) # Different from LoRA

        qlora_cfg = LoraConfig(
            r=rank, lora_alpha=alpha, target_modules=["q_proj", "v_proj"],
            lora_dropout=dropout, task_type="CAUSAL_LM"
        )
        self.model = get_peft_model(model, qlora_cfg)

        args = TrainingArguments(**training_args_kwargs)
        self.trainer = Trainer(
            model=self.model,
            args=args,
            data_collator=DataCollatorForLanguageModeling(self.tokenizer, mlm=False),
            train_dataset=train_dataset,
            eval_dataset=eval_dataset,
        )

    def train(self):
        return self.trainer.train()

    @torch.no_grad()
    def generate(self, prompt: str, max_new_tokens: int = 40):
        device = next(self.model.parameters()).device
        ids = self.tokenizer(prompt, return_tensors="pt").to(device)
        out = self.model.generate(
            **ids,
            max_new_tokens=max_new_tokens,
            pad_token_id=self.tokenizer.eos_token_id
        )
        return self.tokenizer.decode(out[0], skip_special_tokens=True)

    def push_to_hub(self, repo_id: str):
        self.model.push_to_hub(repo_id, use_auth_token=True)
        self.tokenizer.push_to_hub(repo_id, use_auth_token=True)

In [ ]:
MODEL_ID = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
tokenizer.pad_token = tokenizer.eos_token

# --- Toy dataset --- #
examples = [
    {"text": "### User:\nTranslate 'Good morning' to Spanish.\n### Assistant:\n"},
    {"text": "### User:\nSummarise: 'The cat sat on the mat.'\n### Assistant:\n"},
    {"text": "### User:\nList three primary colours.\n### Assistant:\n"},
    {"text": "### User:\nWhat is 2 + 2?\n### Assistant:\n"},
    {"text": "### User:\nRewrite 'I like apples' in the past tense.\n### Assistant:\n"},
]
def tok_fn(e): return tokenizer(
    e["text"], max_length=256, truncation=True, padding="max_length"
)
ds = Dataset.from_list(examples).train_test_split(test_size=0.4, seed=0)
ds_tok = ds.map(tok_fn, remove_columns=["text"])

In [ ]:
lora_ft = QLoRAFineTuner(
    base_model_id=MODEL_ID,
    train_dataset=ds_tok["train"],
    eval_dataset=ds_tok["test"],
    output_dir="./qlora_out",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    learning_rate=2e-4,
    logging_steps=1,
    optim="paged_adamw_8bit",
    report_to=[],
)
lora_ft.train()
print(lora_ft.generate("### User:\nWhat is 2 + 2?\n### Assistant:\n"))